In [1]:
import numpy as np
from scipy.optimize import root_scalar

# ==========================
# Constants from the paper
# ==========================
Tc = 647.096  # [K] Critical temperature of water
pc = 22.064e6  # [Pa] Critical pressure of water

# Molar masses [kg/mol]
M_LiBr = 0.08685
M_H2O = 0.018015268
# Coefficients for pressure equation (Eq. 1)
a = [-2.41303e2,1.91750e7,-1.75521e8,3.25430e7,3.92571e2,-2.12626e3,1.85127e8,1.91216e3]
m = [3, 4, 4, 8, 1, 1, 4, 6]
n = [0, 5, 6, 3, 0, 2, 6, 0]
t = [0, 0, 0, 0, 1, 1, 1, 1]
# Coefficients for water vapor pressure (Eq. 28)
a_ps = [-7.85951783, 1.84408259, -11.7866497,
        22.6807411, -15.9618719, 1.80122502] 
b_ps = [1.0, 1.5, 3.0, 3.5, 4.0, 7.5]
# ===========================================
# Saturation pressure of pure water, p_s(T)
# ===========================================
def ps_water(T):
    T=float(T)
    theta = 1 - T / Tc
    ln_ps = (Tc / T) * sum(a * theta**b for a, b in zip(a_ps, b_ps))
    return pc * np.exp(ln_ps)

# ===========================================
# Pressure equation of LiBr-H2O: p(T, x)
# ===========================================
def p_libr(T, x):
    q = T - sum(
        a[i] * x**m[i] * (0.4 - x)**n[i] * (T / Tc)**t[i]
        for i in range(8)
    )
    return ps_water(q)

# ===========================================
# Solve for x at given T and p
# ===========================================
def find_x_given_T_p(T, p_target):
    T = float(T)
    p_target = float(p_target)
    def objective(x):
        if x <= 0 or x >= 0.74:
            return 1e6
        return p_libr(T, x) - p_target

    a, b = 0.001, 0.73
    fa = objective(a)
    fb = objective(b)

    # Check if a solution is physically possible
    if np.sign(fa) == np.sign(fb):
        return None  # no sign change = no root

    result = root_scalar(objective, bracket=[a, b], method='bisect')

    # ❗ Additional guard to avoid edges
    if result.converged and 0.01 < result.root < 0.72:
        return x_to_w(result.root)
    else:
        return None  # discard if root is on edge



# ===========================================
# Convert molar fraction x → mass fraction w
# ===========================================
def x_to_w(x):
    m1 = x * M_LiBr
    m2 = (1 - x) * M_H2O
    return m1 / (m1 + m2)

from CoolProp.CoolProp import PropsSI

MLiBr = 0.08685      # kg/mol
MH2O = 0.018015      # kg/mol
T_c = 647.096        # K
T_0 = 221.0          # K
h_c = 37548.5        # J/mol

# Coefficients
a_h = [
    2.27431, -7.99511, 385.239, -16394, -422.562,
    0.113314, -8.33474, -17383.3, 6.49763, 3245.52,
    -13464.3, 39932.2, -258877, -0.00193046, 2.80616,
    -40.4479, 145.342, -2.74873, -449.743, -12.1794,
    -0.00583739, 0.233910, 0.341888, 8.85259, -17.8731,
    0.0735179, -0.000179430, 0.00184261, -0.00624282, 0.00684765
]
m_h = [1,1,2,3,6,1,3,5,4,5,5,6,6,1,2,2,2,5,6,7,1,1,2,2,2,3,1,1,1,1]
n_h = [0,1,6,6,2,0,0,4,0,4,5,5,6,0,3,5,7,0,3,1,0,4,2,6,7,0,0,1,2,3]
t_h = [0,0,0,0,0,1,1,1,2,2,2,2,2,3,3,3,3,3,3,3,4,4,4,4,4,4,5,5,5,5]

def mass_to_molar_fraction(w):
    return (w / MLiBr) / (w / MLiBr + (1 - w) / MH2O)

def h_LiBr(T, w):
    x = mass_to_molar_fraction(w)
    h_w_kg = PropsSI('H', 'T', T, 'Q', 0, 'Water')  # J/kg
    h_w_mol = h_w_kg * MH2O                        # J/mol
    second = 0.0
    for i in range(30):
        second += a_h[i] * x**m_h[i] * abs(0.4 - x)**n_h[i] * (T_c / (T - T_0))**t_h[i]
    # Total molar enthalpy [J/mol]
    h_molar = (1 - x) * h_w_mol + h_c * second
    M_mix = x * MLiBr + (1 - x) * MH2O
    h_mass = h_molar / M_mix  # J/kg
    return h_mass

s_c = 79.3933      # J/mol·K
s_ai = [
    1.53091, -4.52564, 698.302, -21666.4, -1475.33,
    0.0847012, -6.59523, -29533.1, 0.00956314, -0.188679,
    9.31752, 5.78104, 13893.1, -17176.2, 415.108,
    -55564.7, -0.00423409, 30.5242, -1.67620, 14.8283,
    0.00303055, -0.0401810, 0.149252, 2.59240, -0.177421,
    -0.0000699650, 0.000605007, -0.00165228, 0.00122966
]
s_mi = [1,1,2,3,6,1,3,5,1,2,2,4,5,5,6,6,1,3,5,7,1,1,1,2,3,1,1,1,1]
s_ni = [0,1,6,6,2,0,0,4,0,0,4,0,4,5,2,5,0,4,0,1,0,2,4,7,1,0,1,2,3]
s_ti = [0,0,0,0,0,1,1,1,2,2,2,2,2,2,2,2,3,3,3,3,4,4,4,4,4,5,5,5,5]

def mass_to_molar_fraction(w):
    return (w / MLiBr) / (w / MLiBr + (1 - w) / MH2O)

def s_LiBr(T, w):
    x = mass_to_molar_fraction(w)

    # Get entropy of pure water (J/kg·K) and convert to J/mol·K
    s_w_kgK = PropsSI('S', 'T', T, 'Q', 0, 'Water')  # J/kg·K
    s_w_molK = s_w_kgK * MH2O                      # J/mol·K
    second = sum(
        ai * x**mi * abs(0.4 - x)**ni * (T_c / (T - T_0))**ti
        for ai, mi, ni, ti in zip(s_ai, s_mi, s_ni, s_ti)
    )

    # Molar entropy of the solution [J/mol·K]
    s_molar = (1 - x) * s_w_molK + s_c * second
    M_mix = x * MLiBr + (1 - x) * MH2O
    s_mass = s_molar / M_mix  # J/kg·K
    return s_mass

from scipy.optimize import root_scalar
def find_T_given_h_x(h_target, x_mass, T_bounds=(273.15, 473.15)):
    """
    Invert the h_LiBr function to solve for T given enthalpy and mass fraction.
    h_target: enthalpy [J/kg]
    x_mass: LiBr mass fraction
    T_bounds: bounds for T in Kelvin
    """
    h_target=float(h_target)
    x_mass=float(x_mass)
    def objective(T):
        try:
            return h_LiBr(T, x_mass) - h_target
        except Exception:
            return 1e6  # fail safe if T is out of range

    result = root_scalar(objective, bracket=T_bounds, method='brentq')
    
    if result.converged:
        return result.root
    else:
        raise ValueError("Temperature solution did not converge.")



In [2]:
from sympy import symbols, Eq, solve

# Define groups of symbols
def solve_system(Temperature_evaporator,P_gascooler,Temperature_after_gascooler,Water_line_temperature,High_pressure_generator_temperature,Pressure_hpg,Low_pressure_generator_temperature,Adsorber_temperature,Condensor_temperature,Efficiency_gas_cooler_1,Efficiency_heat_exchanger_1,Efficiency_heat_exchanger_2,Vars_evaporator_temperature,solution_distribution_ratio):

    solutions=dict()
    misc_equations=T_e_tcrs,n_c,n_t,rp,W_turbine,W_compressor,Q_e_tcrs,COP,efficiency,P2s,s2s,h2s,P_e_tcrs,P5s,s5s,h5s,T5s,Q_gc1,Q_gc2,mw1,mw2,E_gc1,E_shx_1,E_shx_2,T_generator_hpg,P_generator_hpg,Q_generator_hpg,T_generator_lpg,P_generator_lpg,T_adsorber,P_adsorber,Q_adsorber,T_e_vars,P_e_vars,Q_e_vars,T_condensor,P_condensor,Q_condensor,D,w_strong,w_strong_1,w_strong_2,w_strong_3,w_strong_4,w_strong_5,w_weak,m_strong,m_strong_1,m_strong_2,m_strong_3,m_strong_4,m_strong_5,m_ra,m_ra_1,m_ra_2,W_input_tcrs,n_exergy_tcrs,n_exergy_vars,n_exergy_total,COP_tcrs,COP_comb,w_9a,w_9b,m_9a,m_9b,P_9a,P_9b,h_9a,h_9b,s_9a,s_9b,T_9a,T_9b,R,waste_heat,ratio_of_Qe=symbols('T_e_tcrs,n_c,n_t,rp,W_turbine,W_compressor,Q_e_tcrs,COP,efficiency,P2s,s2s,h2s,P_e_tcrs,P5s,s5s,h5s,T5s,' \
    'Q_gc1,Q_gc2,mw1,mw2,E_gc1,E_shx_1,E_shx_2,T_generator_hpg,P_generator_hpg,Q_generator_hpg,T_generator_lpg,P_generator_lpg,T_adsorber,P_adsorber,Q_adsorber,T_e_vars,P_e_vars,Q_e_vars,T_condensor,P_condensor,Q_condensor,D,w_strong,w_strong_1,w_strong_2,w_strong_3,w_strong_4,w_strong_5,w_weak,m_strong,m_strong_1,m_strong_2,m_strong_3,m_strong_4,m_strong_5,m_ra,m_ra_1,m_ra_2,W_input_tcrs,n_exergy_tcrs,n_exergy_vars,n_exergy_total,COP_tcrs,COP_comb,w_9a,w_9b,m_9a,m_9b,P_9a,P_9b,h_9a,h_9b,s_9a,s_9b,T_9a,T_9b,R,waste_heat,ratio_of_Qe')
    P=symbols('P:30')
    h=symbols('h:30')
    x=symbols('x:30')
    w=symbols('w:30')
    s=symbols('s:30')
    m=symbols('m:30')
    T=symbols('T:30')
    all_symbols = T + P +w+ x +s+m +h+(misc_equations)

# Define the equations
    equation = [
        Eq(E_shx_1,Efficiency_heat_exchanger_1),
        Eq(E_shx_2,Efficiency_heat_exchanger_2),
    ]
    solutions.update(solve(equation, all_symbols) )

    #point 1
    equation = [
        Eq(T[1],Temperature_evaporator),
        Eq(T_e_tcrs,Temperature_evaporator),
        Eq(x[1],1),
        Eq(m[1],1)
    ]
    solutions.update(solve(equation, all_symbols) )
    equation=[
        Eq(P[1],PropsSI('P','T',solutions[T[1]],'Q',solutions[x[1]],'CO2')),
        Eq(h[1],PropsSI('H','T',solutions[T[1]],'Q',solutions[x[1]],'CO2')),
        Eq(s[1],PropsSI('S','T',solutions[T[1]],'Q',solutions[x[1]],'CO2')),  
              ]
    solutions.update(solve(equation, all_symbols) )
    
    #point  2s
    equation=[
        Eq(P2s,P_gascooler),
        Eq(s2s, solutions[s[1]])
    ]
    solutions.update(solve(equation, all_symbols))

    equation=[
        Eq(h2s,PropsSI('H','P',solutions[P2s],'S',solutions[s2s],'CO2')),
        #evaporator
        Eq(P_e_tcrs,solutions[P[1]]),
        Eq(rp,solutions[P2s]/solutions[P[1]]),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
         Eq(n_c,0.815+ 0.022*solutions[rp]-0.0041*(solutions[rp]**2)+0.0001*(solutions[rp]**3))
    ]
    solutions.update(solve(equation, all_symbols) )

    #point 2
    equation=[
        Eq(P[2],P_gascooler),
        Eq(m[2],1),
        Eq(h[2],solutions[h[1]]+(solutions[h2s]-solutions[h[1]])/solutions[n_c])
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(T[2],PropsSI('T','P',solutions[P[2]],'H',solutions[h[2]],'CO2')),
        Eq(s[2],PropsSI('S','P',solutions[P[2]],'H',solutions[h[2]],'CO2'))
    ]
    solutions.update(solve(equation, all_symbols) )

      #point 27
    equation=[
        Eq(T[27],Water_line_temperature),
        Eq(x[27],1),
    ]
    solutions.update(solve(equation, all_symbols) )
    
    equation=[
        Eq(h[27],PropsSI('H','T',solutions[T[27]],'Q',solutions[x[27]],'Water')),
        Eq(s[27],PropsSI('S','T',solutions[T[27]],'Q',solutions[x[27]],'Water')),
    ]
    solutions.update(solve(equation, all_symbols) )

    #point 26
    equation=[
        Eq(T[26],Water_line_temperature),
        Eq(x[26],0),
    ]
    solutions.update(solve(equation, all_symbols) )
    
    equation=[
        Eq(h[26],PropsSI('H','T',solutions[T[26]],'Q',solutions[x[26]],'Water')),
        Eq(s[26],PropsSI('S','T',solutions[T[26]],'Q',solutions[x[26]],'Water')),
    ]
    solutions.update(solve(equation, all_symbols) )

    #point 3
    equation=[
        Eq(m[3],1),
        Eq(P[3],P_gascooler),
        Eq(T[3],solutions[T[2]]-Efficiency_gas_cooler_1*(solutions[T[2]]-solutions[T[26]])),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(h[3],PropsSI('H','T',solutions[T[3]],'P',solutions[P[3]],'CO2')),
        Eq(s[3],PropsSI('S','T',solutions[T[3]],'P',solutions[P[3]],'CO2')),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(mw1,(solutions[h[2]]-solutions[h[3]])/(solutions[h[27]]-solutions[h[26]])),
        Eq(m[26],mw1),
        Eq(m[27],mw1),
    ]
    solutions.update(solve(equation, all_symbols) )

    #point 4
    equation=[
        Eq(m[4],1),
        Eq(P[4],P_gascooler),
        Eq(T[4],Temperature_after_gascooler),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(s[4],PropsSI('S','T',solutions[T[4]],'P',solutions[P[4]],'CO2')),
        Eq(h[4],PropsSI('H','T',solutions[T[4]],'P',solutions[P[4]],'CO2')),
    ]
    solutions.update(solve(equation, all_symbols) )

    # point 5s
    equation=[
        Eq(s5s,solutions[s[4]]),
        Eq(P5s,solutions[P_e_tcrs]),
        Eq(T5s,Temperature_evaporator),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(h5s,PropsSI('H','S',solutions[s5s],'P',solutions[P5s],'CO2')),
        # turbine efficiency
        Eq(n_t,0.6)
    ]
    solutions.update(solve(equation, all_symbols) )

    # point 5
    equation=[
        Eq(m[5],1),
        Eq(P[5],solutions[P_e_tcrs]),
        Eq(h[5],solutions[h[4]]-solutions[n_t]*(solutions[h[4]]-solutions[h5s]))
    ]
    solutions.update(solve(equation, all_symbols) )
    
    equation=[
        Eq(T[5],PropsSI('T','P',solutions[P[5]],'H',solutions[h[5]],'CO2')),
        Eq(s[5],PropsSI('S','P',solutions[P[5]],'H',solutions[h[5]],'CO2')),
    ]
    solutions.update(solve(equation, all_symbols) )


    equation=[
        Eq(Q_gc2,solutions[h[3]]-solutions[h[4]]),
        Eq(Q_generator_hpg,solutions[mw1]*(solutions[h[27]]-solutions[h[26]]))
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )


    equation=[
        Eq(T_e_vars,Vars_evaporator_temperature)
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(P_e_vars,PropsSI('P','T',solutions[T_e_vars],'Q',1,'Water')),
        Eq(T_adsorber,Adsorber_temperature),
        Eq(P_adsorber,P_e_vars),
        Eq(T_generator_hpg,High_pressure_generator_temperature),
        Eq(P_generator_hpg,Pressure_hpg),
        Eq(T_generator_lpg,Low_pressure_generator_temperature),
        Eq(T_condensor,Condensor_temperature),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(P_condensor,PropsSI('P','T',solutions[T_condensor],'Q',0,'Water')),
        Eq(P_generator_lpg,P_condensor),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    #point 19
    equation=[
        Eq(T[19],solutions[T_generator_lpg]),
        Eq(P[19],solutions[P_condensor]),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )
    equation=[

        Eq(h[19],PropsSI('H','P',solutions[P[19]],'T',solutions[T[19]],'Water')),
        Eq(s[19],PropsSI('S','P',solutions[P[19]],'T',solutions[T[19]],'Water')),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    #point 15
    equation=[
        Eq(T[15],solutions[T_generator_hpg]),
        Eq(P[15],solutions[P_generator_hpg]),
        Eq(x[15],1)
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )
    equation=[

        Eq(h[15],PropsSI('H','P',solutions[P[15]],'T',solutions[T[15]],'Water')),
        Eq(s[15],PropsSI('S','P',solutions[P[15]],'T',solutions[T[15]],'Water')),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    #point 17
    equation=[
        Eq(P[17],solutions[P_generator_hpg]),
        Eq(x[17],0)
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )
    equation=[

        Eq(T[17],PropsSI('T','P',solutions[P[17]],'Q',solutions[x[17]],'Water')),
        Eq(h[17],PropsSI('H','P',solutions[P[17]],'Q',solutions[x[17]],'Water')),
        Eq(s[17],PropsSI('S','P',solutions[P[17]],'Q',solutions[x[17]],'Water')),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    #point 18
    equation=[
        Eq(P[18],solutions[P_condensor]),
        Eq(h[18],solutions[h[17]])
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    equation=[
        Eq(s[18],PropsSI('S','P',solutions[P[18]],'H',solutions[h[18]],'Water')),
        Eq(T[18],PropsSI('T','P',solutions[P[18]],'H',solutions[h[18]],'Water')),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )
         
    #point 24
    equation=[
        Eq(P[24],solutions[P_condensor]),
        Eq(T[24],solutions[T_condensor])
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    equation=[
        Eq(s[24],PropsSI('S','Q',0,'T',solutions[T[24]],'Water')),
        Eq(h[24],PropsSI('H','Q',0,'T',solutions[T[24]],'Water')),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    #point 25
    equation=[
        Eq(P[25],solutions[P_e_vars]),
        Eq(T[25],solutions[T_e_vars])
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    equation=[
        Eq(h[25],solutions[h[24]]),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    equation=[
        Eq(s[25],PropsSI('S','H',solutions[h[25]],'P',solutions[P[25]],'Water')),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    #point 6
    equation=[
        Eq(P[6],solutions[P_e_vars]),
        Eq(T[6],solutions[T_e_vars])
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    equation=[
        Eq(h[6],PropsSI('H','P',solutions[P_e_vars],'Q',1,"Water")),
        Eq(s[6],PropsSI('S','P',solutions[P_e_vars],'Q',1,"Water")),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    #point 7

    equation=[
        Eq(P[7],solutions[P_adsorber]),
        Eq(T[7],solutions[T_adsorber])
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    equation=[
        Eq(w[7],find_x_given_T_p(solutions[T[7]],solutions[P[7]])),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    equation=[
        Eq(h[7],h_LiBr(solutions[T[7]],solutions[w[7]])),
        Eq(s[7],s_LiBr(solutions[T[7]],solutions[w[7]])),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    equation=[
        Eq(w_strong,solutions[w[7]]),
        Eq(w[8],w_strong),
        Eq(w[9],w_strong),
        Eq(w[9],w_9a),
        Eq(w[9],w_9b),
        Eq(w_strong_1,w_9a),
        Eq(w_strong_2,w_9b),
        Eq(w[10],w_strong_1),
        Eq(w[16],w_strong_2),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    #point 11
    equation=[
        Eq(P[11],solutions[P_generator_hpg]),
        Eq(T[11],solutions[T_generator_hpg])
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    equation=[
        Eq(w[11],find_x_given_T_p(solutions[T[11]],solutions[P[11]])),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    equation=[
        Eq(h[11],h_LiBr(solutions[T[11]],solutions[w[11]])),
        Eq(s[11],s_LiBr(solutions[T[11]],solutions[w[11]])),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )
    
    equation=[
        Eq(w_strong_3,solutions[w[11]]),
        Eq(w_strong_3,w[12]),
        Eq(w_strong_3,w[13]),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    #point 20
    equation=[
        Eq(P[20],solutions[P_generator_lpg]),
        Eq(T[20],solutions[T_generator_lpg])
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    equation=[
        Eq(w[20],find_x_given_T_p(solutions[T[20]],solutions[P[20]])),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    equation=[
        Eq(h[20],h_LiBr(solutions[T[20]],solutions[w[20]])),
        Eq(s[20],s_LiBr(solutions[T[20]],solutions[w[20]])),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    #solution balance
    equation=[
        Eq(D,solution_distribution_ratio),
        Eq(w_strong_4,solutions[w[20]]),
        Eq(w[21],1/((D/solutions[w_strong_3])+((1-D)/w_strong_4))),
        Eq(w_strong_5,w[21]),
        Eq(w[22],w_strong_5),
        Eq(w[23],w_strong_5),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    #point 28
    equation=[
        Eq(T[28],30+273.15),
        Eq(P[0],101325),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    equation=[
        Eq(h[28],PropsSI('H','P',solutions[P[0]],'T',solutions[T[28]],"Water")),
        Eq(s[28],PropsSI('S','P',solutions[P[0]],'T',solutions[T[28]],"Water")),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    #point 29
    equation=[
        Eq(T[29],37+273.15),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    equation=[
        Eq(h[29],PropsSI('H','P',solutions[P[0]],'T',solutions[T[29]],"Water")),
        Eq(s[29],PropsSI('S','P',solutions[P[0]],'T',solutions[T[29]],"Water")),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    #random
    equation=[
        Eq(m[29],solutions[m[3]]*(solutions[h[3]]-solutions[h[4]])/(solutions[h[29]]-solutions[h[28]])),
        Eq(mw2,m[29]),
        Eq(m[28],mw2),
        Eq(h[8],solutions[h[7]]),
        Eq(T[8],solutions[T[7]])
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    equation_block_19=solve_19_equation_block(solutions)
    equation=[
        Eq(m_strong,equation_block_19['m_strong']),
        Eq(m_strong_1,equation_block_19['m_strong_1']),
        Eq(m_strong_2,equation_block_19['m_strong_2']),
        Eq(m_strong_3,equation_block_19['m_strong_3']),
        Eq(m_strong_4,equation_block_19['m_strong_4']),
        Eq(m_strong_5,equation_block_19['m_strong_5']),
        Eq(m_ra_1,equation_block_19['m_ra_1']),
        Eq(m_ra_2,equation_block_19['m_ra_2']),
        Eq(m_ra,equation_block_19['m_ra']),
        Eq(T[9],equation_block_19['T9']),
        Eq(T[12],equation_block_19['T12']),
        Eq(T[21],equation_block_19['T21']),
        Eq(T[22],equation_block_19['T22']),
        Eq(h[9],equation_block_19['h9']),
        Eq(h[10],equation_block_19['h10']),
        Eq(h[12],equation_block_19['h12']),
        Eq(h[13],equation_block_19['h13']),
        # Eq(h[20],equation_block_19['h20']),
        Eq(h[21],equation_block_19['h21']),
        Eq(h[22],equation_block_19['h22']),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )
    
    equation=[
        Eq(m[19],solutions[m_ra_2]),
        Eq(m[15],solutions[m_ra_1]),
        Eq(m[23],solutions[m_strong_5]),
        Eq(m[22],solutions[m_strong_5]),
        Eq(m[21],solutions[m_strong_5]),
        Eq(m_9b,solutions[m_strong_2]),
        Eq(m_9a,solutions[m_strong_1]),
        Eq(m[20],solutions[m_strong_4]),
        Eq(m[16],solutions[m_strong_2]),
        Eq(m[13],solutions[m_strong_3]),
        Eq(m[12],solutions[m_strong_3]),
        Eq(m[11],solutions[m_strong_3]),
        Eq(m[10],solutions[m_strong_1]),
        Eq(m[9],solutions[m_strong]),
        Eq(m[8],solutions[m_strong]),
        Eq(m[7],solutions[m_strong]),
        Eq(m[6],solutions[m_ra]),
        Eq(m[25],solutions[m_ra]),
        Eq(m[24],solutions[m_ra]),
        Eq(m[18],solutions[m_ra_1]),
        Eq(m[17],solutions[m_ra_1]),
        Eq(h_9a,solutions[h[9]]),
        Eq(s[9],s_LiBr(solutions[T[9]],solutions[w[9]])),
        Eq(s_9a,s[9]),
        Eq(T_9a,solutions[T[9]]),
        Eq(h_9b,solutions[h[9]]),
        Eq(s_9b,s[9]),
        Eq(T_9b,solutions[T[9]]),
        Eq(h_9b,h[16]),
        Eq(h[23],solutions[h[22]]),
        Eq(T[23],solutions[T[22]]),
        Eq(T[16],T_9b),
        Eq(T[13],solutions[T[12]]),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    equation=[
        Eq(Q_e_vars,solutions[m[6]]*solutions[h[6]]-solutions[m[25]]*solutions[h[25]]),
        Eq(Q_adsorber,solutions[m[6]]*solutions[h[6]]+solutions[m[23]]*solutions[h[23]]-solutions[m[7]]*solutions[h[7]]),
        Eq(Q_condensor,solutions[m[18]]*solutions[h[18]]+solutions[m[19]]*solutions[h[19]]-solutions[m[24]]*solutions[h[24]]),
        Eq(s[8],s_LiBr(solutions[T[8]],solutions[w[8]])),
        Eq(T[10],find_T_given_h_x(solutions[h[10]],solutions[w[10]])),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    equation=[
        Eq(s[10],s_LiBr(solutions[T[10]],solutions[w[10]])),
        Eq(s[12],s_LiBr(solutions[T[12]],solutions[w[12]])),
        Eq(s[13],s_LiBr(solutions[T[13]],solutions[w[13]])),
        Eq(s[16],s_LiBr(solutions[T[16]],solutions[w[16]])),
        Eq(s[22],s_LiBr(solutions[T[22]],solutions[w[22]])),
        Eq(s[23],s_LiBr(solutions[T[23]],solutions[w[23]])),
        Eq(s[21],(s[13]*solutions[m[13]]+solutions[m[20]]*solutions[s[20]])/solutions[m[21]]),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )
    # print(solutions)
    equation=[
        Eq(T[0],298.15),
        Eq(W_turbine,solutions[h[4]]-solutions[h[5]]),
        Eq(W_compressor,solutions[h[2]]-solutions[h[1]]),
        Eq(W_input_tcrs,W_compressor-W_turbine),
        Eq(Q_e_tcrs,solutions[h[1]]-solutions[h[5]]),
        Eq(COP,(solutions[Q_e_vars]+Q_e_tcrs)/W_input_tcrs),
        Eq(n_exergy_total,((Q_e_tcrs*(T[0]/(solutions[T_e_tcrs]+5)-1))+solutions[Q_e_vars]*(T[0]/(solutions[T_e_vars]+5)-1))/W_input_tcrs),
        Eq(R,waste_heat/W_input_tcrs),
        Eq(waste_heat,solutions[Q_gc2]+solutions[Q_condensor]+solutions[Q_adsorber]),
        Eq(ratio_of_Qe,Q_e_vars/Q_e_tcrs)
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )
    return solutions


In [4]:
solve_system(-25+273.15,13000000,40+273.15,145+273.15,273.15+140,80000,80+273.15,35+273.15,273.15+35,0.8,0.7,0.7,7+273.15,0.1)

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\optimize\_zeros_py.py:577: ComplexWarning: Casting complex values to real discards the imaginary part
  r = _zeros._bisect(f, a, b, xtol, rtol, maxiter, args, full_output, disp)


{E_shx_1: 0.700000000000000,
 E_shx_2: 0.700000000000000,
 T1: 248.150000000000,
 T_e_tcrs: 248.150000000000,
 m1: 1.00000000000000,
 x1: 1.00000000000000,
 P1: 1682738.12720116,
 h1: 437055.401286796,
 s1: 1973.20884889544,
 P2s: 13000000.0000000,
 s2s: 1973.20884889544,
 P_e_tcrs: 1682738.12720116,
 h2s: 535874.827462082,
 rp: 7.72550392117308,
 n_c: 0.786367544282256,
 P2: 13000000.0000000,
 h2: 562721.099081435,
 m2: 1.00000000000000,
 T2: 427.485906678543,
 s2: 2037.47890798093,
 T27: 418.150000000000,
 x27: 1.00000000000000,
 h27: 2739799.55404785,
 s27: 6882.60864468998,
 T26: 418.150000000000,
 x26: 0.0,
 h26: 610638.392919176,
 s26: 1790.74887874801,
 P3: 13000000.0000000,
 T3: 420.017181335709,
 m3: 1.00000000000000,
 h3: 552485.167654763,
 s3: 2013.32194401378,
 m26: 0.00480749490153481,
 m27: 0.00480749490153481,
 mw1: 0.00480749490153481,
 P4: 13000000.0000000,
 T4: 313.150000000000,
 m4: 1.00000000000000,
 h4: 291614.625917768,
 s4: 1274.06093058992,
 P5s: 1682738.1272011

In [3]:
import numpy as np
from scipy.optimize import fsolve
from CoolProp.CoolProp import PropsSI

MLiBr = 0.08685      # kg/mol
MH2O = 0.018015      # kg/mol
T_c = 647.096        # K
T_0 = 221.0          # K
h_c = 37548.5        # J/mol

# Coefficients
a_h = [
    2.27431, -7.99511, 385.239, -16394, -422.562,
    0.113314, -8.33474, -17383.3, 6.49763, 3245.52,
    -13464.3, 39932.2, -258877, -0.00193046, 2.80616,
    -40.4479, 145.342, -2.74873, -449.743, -12.1794,
    -0.00583739, 0.233910, 0.341888, 8.85259, -17.8731,
    0.0735179, -0.000179430, 0.00184261, -0.00624282, 0.00684765
]
m_h = [1,1,2,3,6,1,3,5,4,5,5,6,6,1,2,2,2,5,6,7,1,1,2,2,2,3,1,1,1,1]
n_h = [0,1,6,6,2,0,0,4,0,4,5,5,6,0,3,5,7,0,3,1,0,4,2,6,7,0,0,1,2,3]
t_h = [0,0,0,0,0,1,1,1,2,2,2,2,2,3,3,3,3,3,3,3,4,4,4,4,4,4,5,5,5,5]

def mass_to_molar_fraction(w):
    return (w / MLiBr) / (w / MLiBr + (1 - w) / MH2O)

def h_librh2o(T, w):
    x = mass_to_molar_fraction(w)
    h_w_kg = PropsSI('H', 'T', T, 'Q', 0, 'Water')  # J/kg
    h_w_mol = h_w_kg * MH2O                        # J/mol
    second = 0.0
    for i in range(30):
        second += a_h[i] * x**m_h[i] * abs(0.4 - x)**n_h[i] * (T_c / (T - T_0))**t_h[i]
    # Total molar enthalpy [J/mol]
    h_molar = (1 - x) * h_w_mol + h_c * second
    M_mix = x * MLiBr + (1 - x) * MH2O
    h_mass = h_molar / M_mix  # J/kg
    return h_mass
def solve_19_equation_block(values):
    def equations(vars):
        # Unpack all variables
        (m_strong, m_strong_1, m_strong_2, m_strong_3, m_strong_4, m_strong_5, m_ra_1, m_ra_2, m_ra,
        T9, T12, T21, T22, h9, h10, h12, h13, h20, h21, h22) = vars
        solutions = {str(k): v for k, v in values.items()}
        # Given constants (converted to SI units where needed)
        h11 = solutions["h11"]    # J/kg
        h15 = solutions["h15"]      # J/kg
        Q_generator_hpg = solutions["Q_generator_hpg"]    # kW → W? Assuming it's already in consistent units
        h19 = solutions["h19"]    # J/kg
        h17 = solutions["h17"]      # J/kg
        h20_given = solutions["h20"] # J/kg
        h8 = solutions["h8"]        # J/kg
        E_shx_1 = solutions["E_shx_1"]
        E_shx_2 = solutions["E_shx_2"]
        T8 = solutions["T8"]          # K
        T11 =  solutions["T11"]          # K
        X9 = solutions["w9"]
        X21 = solutions["w21"]
        X22 = solutions["w22"]
        X12 = solutions["w12"]
        X7 = solutions["w7"]
        D = solutions['D']
        X23 = solutions["w23"]
        
        # Initialize the equations array
        eqs = []
        
        # Equation 1: X7*m_s = X23*m_s5
        eqs.append(X7 * m_strong - X23 * m_strong_5)
        
        # Equation 2: m_s1 = m_s3 + m_ra1
        eqs.append(m_strong_1 - m_strong_3 - m_ra_1)
        
        # Equation 3: m_s3*h11 + m_ra1*h15 = Q_hpg + m_s1*h10
        eqs.append(m_strong_3*h11 + m_ra_1*h15 - Q_generator_hpg - m_strong_1*h10)
        
        # Equation 4: m_s2 = m_ra2 + m_s4
        eqs.append(m_strong_2 - m_ra_2 - m_strong_4)
        
        # Equation 5: m_s2*h9 + m_ra1*h15 = m_ra2*h19 + m_ra1*h17 + m_s4*h20
        eqs.append(m_strong_2*h9 + m_ra_1*h15 - m_ra_2*h19 - m_ra_1*h17 - m_strong_4*h20)
        
        # Equation 6: m_ra2 + m_ra1 = m_ra
        eqs.append(m_ra_2 + m_ra_1 - m_ra)
        
        # Equation 7: m_s*(h9 - h8) = m_s5*(h21 - h22)
        eqs.append(m_strong*(h9 - h8) - m_strong_5*(h21 - h22))
        
        # Equation 8: E_shx1 = (T21 - T22)/(T21 - T8)
        eqs.append(E_shx_1 - (T21 - T22)/(T21 - T8))
        
        # Equation 9: m_s1*(h10 - h9) = m_s3*(h11 - h12)
        eqs.append(m_strong_1*(h10 - h9) - m_strong_3*(h11 - h12))
        
        # Equation 10: E_shx2 = (T11 - T12)/(T11 - T9)
        eqs.append(E_shx_2 - (T11 - T12)/(T11 - T9))
        
        # Equation 11: m_s = m_s1 + m_s2
        eqs.append(m_strong - m_strong_1 - m_strong_2)
        
        # Equation 12: m_s5 = m_s3 + m_s4
        eqs.append(m_strong_5 - m_strong_3 - m_strong_4)
        
        # Equation 13: m_s1 = D*m_s
        eqs.append(m_strong_1 - D*m_strong)
        
        # Equation 14: m_s5*h21 = m_s3*h13 + m_s4*h20
        eqs.append(m_strong_5*h21 - m_strong_3*h13 - m_strong_4*h20)
        
        # Equation 15: h9 = h_librh2o(T9, X9)
        eqs.append(h9 - h_librh2o(T9, X9))
        
        # Equation 16: h21 = h_librh2o(T21, X21)
        eqs.append(h21 - h_librh2o(T21, X21))
        
        # Equation 17: h22 = h_librh2o(T22, X22)
        eqs.append(h22 - h_librh2o(T22, X22))
        
        # Equation 18: h12 = h_librh2o(T12, X12)
        eqs.append(h12 - h_librh2o(T12, X12))
        
        # Equation 19: h13 = h12
        eqs.append(h13 - h12)
        
        # Equation 20: h20 = h20_given
        eqs.append(h20 - h20_given)
        
        return eqs

    # Initial guesses for all variables
    # These should be based on reasonable estimates of your system
    initial_guess = [
        0.1,       # m_s (kg/s)
        0.1,       # m_s1 (kg/s)
        0.9,       # m_s2 (kg/s)
        0.05,      # m_s3 (kg/s)
        0.85,      # m_s4 (kg/s)
        0.9,       # m_s5 (kg/s)
        0.05,      # m_ra1 (kg/s)
        0.05,      # m_ra2 (kg/s)
        0.1,       # m_ra (kg/s)
        308.15,    # T9 (K) - slightly above T8
        373.15,    # T12 (K) - between T9 and T11
        373.15,    # T21 (K)
        323.15,    # T22 (K)
        100000,    # h9 (J/kg)
        150000,    # h10 (J/kg)
        120000,    # h12 (J/kg)
        120000,    # h13 (J/kg)
        195831.93, # h20 (J/kg) - fixed value
        300000,    # h21 (J/kg)
        200000     # h22 (J/kg)
    ]

    # Solve the system of equations
    sol = fsolve(equations, initial_guess)

    # Extract the solution
    (m_s, m_s1, m_s2, m_s3, m_s4, m_s5, m_ra1, m_ra2, m_ra,
    T9, T12, T21, T22, h9, h10, h12, h13, h20, h21, h22) = sol
    solution_dict={
        "m_strong":sol[0],
        "m_strong_1":sol[1],
        "m_strong_2":sol[2],
        "m_strong_3":sol[3],
        "m_strong_4":sol[4],
        "m_strong_5":sol[5],
        "m_ra_1":sol[6],
        "m_ra_2":sol[7],
        "m_ra":sol[8],
        "T9":sol[9],
        "T12":sol[10],
        "T21":sol[11],
        "T22":sol[12],
        "h9":sol[13],
        "h10":sol[14],
        "h12":sol[15],
        "h13":sol[16],
        "h20":sol[17],
        "h21":sol[18],
        "h22":sol[19],
    }
    return solution_dict


